In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135262")
exp = Experiment(workspace=ws, name="AutoMLProjExp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FVD89UEKV to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-135262
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-135262


In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cpu_cluster_name = 'MLProjCompute'
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig


# Specify parameter sampler
ps = RandomParameterSampling( 
    {
        "--max_iter": choice(10,50,100,150,200)
        ,"--C": choice(0.5,0.8,0.9,1,1.25,1.5)
    }
)
        
# Specify a Policy
# Specify parameter sampler
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')
# src = ScriptRunConfig(source_directory='.', script='train.py', arguments=['--kernel', 'linear', '--penalty', 1.0], environment=sklearn_env)
src = ScriptRunConfig(source_directory='.', script='train.py', environment=sklearn_env)
src.run_config.target = compute_target

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4)
print(hyperdrive_config)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core import Experiment, Workspace

# run = Experiment(ws,'train2').submit(src)
# run.wait_for_completion(show_output=True)

experiment = Experiment(ws, exp.name)
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

RunId: HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9
Web View: https://ml.azure.com/experiments/AutoMLProjExp/runs/HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-135262/workspaces/quick-starts-ws-135262

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-18T20:46:21.149178][API][INFO]Experiment created<END>\n""<START>[2021-01-18T20:46:21.636068][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-18T20:46:21.804903][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-18T20:46:23.0467755Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [5]:
import joblib
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

for child_run in hyperdrive_run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print("Best run metrics :",best_run.get_metrics())
print('Accuracy:', best_run_metrics['Accuracy'])
print("Best run details :",best_run.get_details())
print('Regularization Rate:',parameter_values)
print("Best run file names :",best_run.get_file_names())


# Register model
best_run.register_model(model_path='outputs/dpbanktrain.pkl', model_name='dpbanktrain',
                        tags={'Training context':'Hyperdrive'},
                        properties={'Accuracy': best_run_metrics['Accuracy']})



# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_29', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.9078907435508345, 'status': 'Completed'}
{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_21', 'hyperparameters': '{"--C": 1.25, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_23', 'hyperparameters': '{"--C": 0.9, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_12', 'hyperparameters': '{"--C": 0.8, "--max_iter": 200}', 'best_primary_metric': 0.9072837632776934, 'status': 'Completed'}
{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_17', 'hyperparameters': '{"--C": 0.9, "--max_iter": 150}', 'best_primary_metric': 0.9071320182094081, 'status': 'Completed'}
{'run_id': 'HD_6606dab1-d34f-429d-8535-6b44ff6a8ab9_14', 'hyperparameters': '{"--C": 0.5, "--max_iter": 200}', 'best

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_url_path)

In [8]:


from azureml.core import Dataset
import pandas as pd

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    
    return x_df, y_df

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

ds_automl = Dataset.get_by_name(workspace=ws, name="Bank-marketing", version='latest')
train_data, test_data = ds_automl.random_split(percentage=0.5, seed=50)

In [9]:

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=20,
    task='classification',
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5
)


In [10]:
# Submit your automl run

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, "AutoMLProjExp")
run_auto = experiment.submit(config=automl_config, show_output=True)

RunDetails(run_auto).show()


Running on remote.
No run_configuration provided, running on MLProjCompute with default configuration
Running on remote compute: MLProjCompute
Parent Run ID: AutoML_f6431fa8-cd51-40d7-817e-97ca693a2e1d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towar

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
best_run, fitted_model = run_auto.get_output()

print(best_run)
print(fitted_model)

# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Run(Experiment: AutoMLProjExp,
Id: AutoML_f6431fa8-cd51-40d7-817e-97ca693a2e1d_4,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_decrease=0.0,
                                                                                                    min_impurity_split=None,
              

In [12]:
best_run.get_metrics()

{'average_precision_score_macro': 0.824450322012354,
 'weighted_accuracy': 0.963726345909129,
 'AUC_macro': 0.9476486918872478,
 'f1_score_weighted': 0.9087307426292842,
 'precision_score_macro': 0.8061783689981799,
 'balanced_accuracy': 0.7217668402021981,
 'accuracy': 0.915585873177552,
 'f1_score_macro': 0.7541490436495204,
 'f1_score_micro': 0.915585873177552,
 'average_precision_score_weighted': 0.9554164246696105,
 'average_precision_score_micro': 0.9813202482016592,
 'matthews_correlation': 0.5207309483820592,
 'AUC_micro': 0.9805680532817233,
 'log_loss': 0.17861742829013716,
 'AUC_weighted': 0.9476486918872478,
 'recall_score_micro': 0.915585873177552,
 'recall_score_weighted': 0.915585873177552,
 'precision_score_weighted': 0.9068791013829693,
 'norm_macro_recall': 0.4435336804043962,
 'precision_score_micro': 0.915585873177552,
 'recall_score_macro': 0.7217668402021981,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_f6431fa8-cd51-40d7-817e-97ca693a2e1d_4/co

In [13]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoMLProjExp,AutoML_f6431fa8-cd51-40d7-817e-97ca693a2e1d_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
compute_target.delete()